# Testing Data Collection Methods

This notebook aims to test the data collection methods for the ETL Pipeline. It will cover:

- Fetching Weather Data from Meteostat
- Fetching Zonal Price Data
- Fetching Load and Demand Data
- Fetching Generation Data
- Fetching Transmission and Interchange Data
- Fetching Reserve Margins

We will be testing this with the Florida Region of US-FLA-FMPP, with Latitude: 28.525581, Longitude: -81.536775, Altitude: 0

## Import Libraries and API Keys

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
import pytz
from meteostat import Point, Hourly, Normals
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set up API keys from environment variables
EIA_API_KEY = os.getenv("EIA_API_KEY")
OPENEI_API_KEY = os.getenv("OPENEI_API_KEY")
NREL_API_KEY = os.getenv("NREL_API_KEY")

# Base URLs
EIA_BASE_URL = "https://api.eia.gov/v2"
OPENEI_BASE_URL = "https://api.openei.org/utility_rates"
NREL_BASE_URL = "https://developer.nrel.gov/api/utility_rates/v3"

In [2]:
EIA_API_KEY

'nHNu2jtfHe4MRbgJJd1Ir3ICh7GThKnDbT8D90WB'

## Interpolation and Forecasting Functions

In [18]:
def interpolate_to_15min(df, timestamp_column='time'):
    """
    Interpolates a DataFrame to 15-minute frequency
    
    Parameters:
        df (pd.DataFrame): Input DataFrame with time series data
        timestamp_column (str): Name of the column containing timestamps
        
    Returns:
        pd.DataFrame: DataFrame with 15-minute frequency data
    """
    if df.empty:
        return df
    
    # Make a copy to avoid modifying the original dataframe
    df_copy = df.copy()
    
    # Ensure the timestamp column is a datetime type
    if not pd.api.types.is_datetime64_any_dtype(df_copy[timestamp_column]):
        df_copy[timestamp_column] = pd.to_datetime(df_copy[timestamp_column])
    
    # Save non-numeric columns before setting index
    non_numeric_columns = [col for col in df_copy.columns 
                          if col != timestamp_column and not pd.api.types.is_numeric_dtype(df_copy[col])]
    
    # Set the timestamp column as index if it's not already
    if df_copy.index.name != timestamp_column:
        df_copy = df_copy.set_index(timestamp_column)
    
    # Get start and end times from the data
    start_time = df_copy.index.min()
    end_time = df_copy.index.max()
    
    # Create a new DatetimeIndex with 15-minute frequency
    new_index = pd.date_range(start=start_time, end=end_time, freq='15min')
    
    # Reindex the DataFrame - this will create NaN for all new timestamps
    df_reindexed = df_copy.reindex(new_index)
    
    # Handle non-numeric columns separately
    non_numeric_data = {}
    for col in non_numeric_columns:
        if col in df_copy.columns:
            # Extract non-numeric column and forward fill after reindexing
            non_numeric_data[col] = df_copy[col].reindex(new_index).ffill().bfill()
            # Remove from the main dataframe to avoid conflict with interpolation
            if col in df_reindexed.columns:
                df_reindexed = df_reindexed.drop(col, axis=1)
    
    # Apply interpolation method (linear by default) to numeric columns
    df_interpolated = df_reindexed.interpolate(method='linear')
    
    # Add back the non-numeric columns
    for col, data in non_numeric_data.items():
        df_interpolated[col] = data
    
    # Reset index to make the timestamp a column again
    df_interpolated = df_interpolated.reset_index()
    df_interpolated.rename(columns={'index': timestamp_column}, inplace=True)
    
    return df_interpolated

## Weather Data

In [19]:
def get_weather_data(zone_id, start_date, end_date):
    """
    Fetches weather data for a specific zone using meteostat
    
    Parameters:
        zone_id (str): Zone identifier (e.g., 'US-FLA-FMPP')
        start_date (datetime): Start date for the data
        end_date (datetime): End date for the data
        
    Returns:
        pd.DataFrame: DataFrame with weather data interpolated to 15-minute intervals
    """
    # Define a mapping of zone_ids to coordinates (lat, lon, alt)
    zone_coords = {
        'US-FLA-FMPP': (28.525581, -81.536775, 0),
        'US-FLA-FPC': (28.996695, -82.886613, 0),
        'US-FLA-FPL': (27.917488, -81.450970, 0),
        'US-FLA-GVL': (29.619310, -82.328732, 0),
        'US-FLA-HST': (25.456904, -80.588092, 0),
        'US-FLA-JEA': (30.390902, -83.679837, 0),
        'US-FLA-SEC': (28.805983, -82.306291, 0),
        'US-FLA-TAL': (30.437174, -84.248042, 0),
        'US-FLA-TEC': (27.959413, -82.144821, 0),
        'US-SE-SOCO': (32.154182, -85.283967, 0),
        'US-CAR-CPLE': (35.299120, -77.893222, 0),
        'US-CAR-CPLW': (35.602668, -82.602688, 0),
        'US-CAR-DUK': (35.222090, -81.841604, 0),
        'US-CAR-SC': (34.043817, -80.365381, 0),
        'US-CAR-SCEG': (33.514198, -81.052635, 0),
        'US-MIDW-AECI': (37.574031, -93.321490, 0),
        'US-MIDW-LGEE': (37.589836, -85.663064, 0),
        'US-MIDW-MISO': (40.211376, -90.411633, 0),
        'US-TEN-TVA': (35.166141, -86.702414, 0)
    }
    
    # Get coordinates for the zone
    if zone_id not in zone_coords:
        raise ValueError(f"Zone {zone_id} not found in coordinate mapping")
    
    lat, lon, alt = zone_coords[zone_id]
    
    # Create a Point for the location
    location = Point(lat, lon, alt)
    
    # Ensure start_date and end_date are datetime objects
    if not isinstance(start_date, datetime):
        start_date = pd.to_datetime(start_date)
    if not isinstance(end_date, datetime):
        end_date = pd.to_datetime(end_date)
    
    # Fetch hourly data
    data = Hourly(location, start_date, end_date)
    hourly_data = data.fetch()
    
    # Reset index to make time a column
    hourly_data = hourly_data.reset_index()
    
    # Handle NaN values in specific columns
    
    # snow (Snow depth) - For southern states, typically safe to assume 0
    if 'snow' in hourly_data.columns:
        # For Florida and most southern states, snow is extremely rare
        if zone_id.startswith('US-FLA') or zone_id.startswith('US-SE') or zone_id.startswith('US-CAR'):
            hourly_data['snow'] = hourly_data['snow'].fillna(0)
        else:
            # For other regions, use seasonal logic
            def fill_snow(row):
                month = row['time'].month
                # If summer months (May-Sep), fill with 0
                if 5 <= month <= 9:
                    return 0
                # Otherwise keep as NaN for later interpolation
                return row['snow']
            
            hourly_data['snow'] = hourly_data.apply(fill_snow, axis=1)
    
    # wpgt (Peak wind gust) - Can estimate from wind speed
    if 'wpgt' in hourly_data.columns and 'wspd' in hourly_data.columns:
        # Estimate peak gusts as 1.5x the sustained wind speed
        hourly_data['wpgt'] = hourly_data.apply(
            lambda row: row['wpgt'] if not pd.isna(row['wpgt']) else row['wspd'] * 1.5 if not pd.isna(row['wspd']) else None, 
            axis=1
        )
    
    # tsun (Sunshine duration) - Harder to estimate, but can use daylight hours
    if 'tsun' in hourly_data.columns:
        # Fill nighttime hours with 0
        hourly_data['tsun'] = hourly_data.apply(
            lambda row: 0 if (row['time'].hour < 6 or row['time'].hour > 18) else row['tsun'],
            axis=1
        )
        
        # For remaining NaN during daytime, use cloud cover if available
        if 'coco' in hourly_data.columns:
            hourly_data['tsun'] = hourly_data.apply(
                lambda row: (1 - row['coco']/100) * 60 if pd.isna(row['tsun']) and 6 <= row['time'].hour <= 18 and not pd.isna(row['coco']) else row['tsun'],
                axis=1
            )
    
    # Convert temperature from Celsius to Fahrenheit
    if 'temp' in hourly_data.columns:
        hourly_data['temp_f'] = hourly_data['temp'] * 9/5 + 32
    
    # Convert wind speed from km/h to mph
    if 'wspd' in hourly_data.columns:
        hourly_data['wspd_mph'] = hourly_data['wspd'] * 0.621371
    
    # For any remaining NaN values, forward fill then backward fill
    hourly_data = hourly_data.ffill().bfill()
    
    # Interpolate to 15-minute intervals
    df_15min = interpolate_to_15min(hourly_data, 'time')
    
    return df_15min

In [20]:
get_weather_data('US-FLA-FMPP', datetime(2024, 1, 1), datetime(2024, 1, 31))

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,temp_f,wspd_mph
0,2024-01-01 00:00:00,14.200,8.800,70.00,0.0,0.0,3.00,9.00,13.500,1024.000,0.0,1.0,57.560,5.592339
1,2024-01-01 00:15:00,14.300,8.900,70.00,0.0,0.0,2.25,6.75,10.125,1024.025,0.0,1.0,57.740,4.194254
2,2024-01-01 00:30:00,14.400,9.000,70.00,0.0,0.0,1.50,4.50,6.750,1024.050,0.0,1.0,57.920,2.796169
3,2024-01-01 00:45:00,14.500,9.100,70.00,0.0,0.0,0.75,2.25,3.375,1024.075,0.0,1.0,58.100,1.398085
4,2024-01-01 01:00:00,14.600,9.200,70.00,0.0,0.0,0.00,0.00,0.000,1024.100,0.0,1.0,58.280,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876,2024-01-30 23:00:00,16.300,5.300,48.00,0.0,0.0,290.00,7.60,11.400,1019.700,0.0,3.0,61.340,4.722420
2877,2024-01-30 23:15:00,15.875,5.525,50.25,0.0,0.0,290.00,8.95,13.425,1019.725,0.0,3.0,60.575,5.561270
2878,2024-01-30 23:30:00,15.450,5.750,52.50,0.0,0.0,290.00,10.30,15.450,1019.750,0.0,3.0,59.810,6.400121
2879,2024-01-30 23:45:00,15.025,5.975,54.75,0.0,0.0,290.00,11.65,17.475,1019.775,0.0,3.0,59.045,7.238972


## Zonal Price Data

## Load and Demand Data

## Generation Data

## Transmission and Interchange Data

## Reserve Margins